In [1]:
import vamp
import os
import sys
import pickle
import numpy as np
import essentia.standard as ess
import essentia.streaming as esstr
import essentia

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

%matplotlib inline
import IPython.display as ipd
from pygments import highlight
from pygments.lexers import JavascriptLexer
from pygments.formatters import HtmlFormatter
import IPython
import pandas as pd
import seaborn as sns

from sklearn.mixture import GaussianMixture
import mir_eval.sonify as sonify


def nnlsChromaFromAudio(audiofile, sampleRate=44100, stepSize=2048):
    mywindow = np.array(
        [0.001769, 0.015848, 0.043608, 0.084265, 0.136670, 0.199341, 0.270509, 0.348162, 0.430105, 0.514023,
         0.597545, 0.678311, 0.754038, 0.822586, 0.882019, 0.930656, 0.967124, 0.990393, 0.999803, 0.999803,
         0.999803, 0.999803, 0.999803, 0.999803, 0.999803, 0.999803, 0.999803, 0.999803, 0.999803, 0.999803,
         0.999803, 0.999803, 0.999803, 0.999803, 0.999803,
         0.999803, 0.999803, 0.999803, 0.999803, 0.999803, 0.999803, 0.999803, 0.999650, 0.996856, 0.991283,
         0.982963, 0.971942, 0.958281, 0.942058, 0.923362, 0.902299, 0.878986, 0.853553, 0.826144,
         0.796910, 0.766016, 0.733634, 0.699946, 0.665140, 0.629410, 0.592956, 0.555982, 0.518696,
         0.481304, 0.444018, 0.407044, 0.370590, 0.334860, 0.300054, 0.266366, 0.233984, 0.203090,
         0.173856, 0.146447, 0.121014, 0.097701, 0.076638, 0.057942, 0.041719, 0.028058, 0.017037,
         0.008717, 0.003144, 0.000350])
    audio = essentia.standard.MonoLoader(filename=audiofile, sampleRate=sampleRate)()

    stepsize, semitones = vamp.collect(
        audio, sampleRate, "nnls-chroma:nnls-chroma", output="semitonespectrum", step_size=stepSize)["matrix"]
    chroma = np.zeros((semitones.shape[0], 12))
    for i in range(semitones.shape[0]):
        tones = semitones[i] * mywindow
        cc = chroma[i]
        for j in range(tones.size):
            cc[j % 12] = cc[j % 12] + tones[j]
    # roll from 'A' based to 'C' based
    chroma = np.roll(chroma, shift=-3, axis=1)
    return chroma

In [2]:
chroma = nnlsChromaFromAudio('../data/chroma/maj35fromE.wav')

In [3]:
chroma

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   9.51086413e-02, ...,
          1.08412935e-04,   7.38242965e-04,   3.53295480e+00],
       [  0.00000000e+00,   2.65900522e-02,   3.42743365e-01, ...,
          1.12759329e-04,   1.31285274e-03,   3.18792227e+00],
       [  5.92510718e-01,   1.90285606e-01,   6.98561950e-01, ...,
          1.98473061e-04,   2.79598946e-03,   2.35216722e+00]])